In [66]:
# Dependencies
import numpy as np
import pandas as pd
import requests
import json
from pprint import pprint
import time
from datetime import timedelta,datetime,date

# Libraries to support access to the  Census API wrapper
from census import Census
from config import census_api_key as api_key
from us import states

# Import Census Data

In [67]:
#setting an API object for the census data, with the desired year; 2016 is the latest data available at this level of detail from teh ACS5
c = Census(api_key, year = 2016)

#Setting the path for a sheet that contains codes and mapping for different categories of Census data
census_terms_path = ('Census_search_terms.xlsx')

#Creating dataframes from the XLSX which contain the codes for 59 different metrics and the FIPS state codes
census_terms_df = pd.read_excel(census_terms_path, sheetname = "Sheet1")
fips_codes_df = pd.read_excel(census_terms_path, sheetname = "Sheet2", dtype = str)

In [4]:
#grabbing census data from the Census API and putting into a list
census_data = []

for row in census_terms_df['code']:
    try:
        acs5_data = c.acs5.state(row, Census.ALL)
        census_data.append(acs5_data)
        print(f"grabbing {row}")
    except:
        print(f"grabbing {row} failed")

grabbing B01001_001E
grabbing B01001_002E
grabbing B01001_026E
grabbing B01002_001E
grabbing B01002_002E
grabbing B01002_003E
grabbing B23025_001E
grabbing B23025_002E
grabbing B23025_003E
grabbing B23025_004E
grabbing B23025_005E
grabbing B23025_006E
grabbing B23025_007E
grabbing B06009_001E
grabbing B06009_002E
grabbing B06009_003E
grabbing B06009_004E
grabbing B06009_005E
grabbing B06009_006E
grabbing B06010_001E
grabbing B06010_002E
grabbing B06010_003E
grabbing B06010_004E
grabbing B06010_005E
grabbing B06010_006E
grabbing B06010_007E
grabbing B06010_008E
grabbing B06010_009E
grabbing B06010_010E
grabbing B06010_011E
grabbing B08122_001E
grabbing B08122_005E
grabbing B08122_009E
grabbing B08122_013E
grabbing B08122_017E
grabbing B08122_021E
grabbing B08122_025E
grabbing B08124_001E
grabbing B08124_002E
grabbing B08124_003E
grabbing B08124_004E
grabbing B08124_005E
grabbing B08124_006E
grabbing B08124_007E
grabbing B08126_001E
grabbing B08126_002E
grabbing B08126_003E
grabbing B081

In [68]:
#taking all data from the list and adding it together into a single list
#then create a dataframe out of each and appending together

for i,data in enumerate(census_data):
    if i == 0:
        all_census_data_df = pd.DataFrame(data)
        all_census_data_df.set_index('state', inplace = True)
    else:
        all_census_data_df_2 = pd.DataFrame(data)
        all_census_data_df_2.set_index('state', inplace = True)
        all_census_data_df = all_census_data_df.merge(all_census_data_df_2, left_index = True, right_index = True)

In [69]:
#create a dictionary with variable keys as the census code and variables as the variable names
columns = dict(zip(census_terms_df['code'], census_terms_df['var_name']))

In [70]:
#rename columns to the human friendly variable names & then reset the index
all_census_data_df = all_census_data_df.rename(columns = columns)
all_census_data_df.reset_index(inplace = True)

#check out the dataset
all_census_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 60 columns):
state                                          52 non-null object
tot_pop                                        52 non-null object
male_pop                                       52 non-null float64
female_pop                                     52 non-null float64
median_age                                     52 non-null float64
median_age_male                                52 non-null float64
median_age_female                              52 non-null float64
employ_status_tot                              52 non-null float64
employ_status_working                          52 non-null float64
employ_status_working_civ                      52 non-null float64
employ_status_working_civ_employed             52 non-null float64
employ_status_working_civ_unemployed           52 non-null float64
employ_status_working_military                 52 non-null float64
employ_status_not_working   

In [71]:
#merge the state codes into the list so that we have a human readable State
all_census_data_df_with_states = pd.merge(all_census_data_df, fips_codes_df, left_on = 'state', right_on = "FIPS", how = "inner")

In [72]:
#Perform some cleansing on the file to remove unwanted columns and to ensure that one of population metrics is numeric data
all_census_data_df_with_states.drop('state', axis = 1, inplace = True)
all_census_data_df_with_states['tot_pop'] = all_census_data_df_with_states['tot_pop'].apply(pd.to_numeric)

#recheck the data
all_census_data_df_with_states.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52 entries, 0 to 51
Data columns (total 62 columns):
tot_pop                                        52 non-null int64
male_pop                                       52 non-null float64
female_pop                                     52 non-null float64
median_age                                     52 non-null float64
median_age_male                                52 non-null float64
median_age_female                              52 non-null float64
employ_status_tot                              52 non-null float64
employ_status_working                          52 non-null float64
employ_status_working_civ                      52 non-null float64
employ_status_working_civ_employed             52 non-null float64
employ_status_working_civ_unemployed           52 non-null float64
employ_status_working_military                 52 non-null float64
employ_status_not_working                      52 non-null float64
birth_ed_tot                

In [73]:
all_census_data_df_with_states['State'].unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri',
       'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey',
       'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio',
       'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island',
       'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah',
       'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin',
       'Wyoming', 'Puerto Rico'], dtype=object)

# Acquire risk factors data

In [74]:
#Setting the path for a sheet that contains risk factor data
risk_path = ('BRFSS_2014_Overall.xlsx')

#Creating dataframes from the CSV which contains the risk data
risk_df = pd.read_excel(risk_path, sheetname = "BRFSS_2014_Overall")

In [75]:
risk_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9295 entries, 0 to 9294
Data columns (total 27 columns):
Year                          9295 non-null int64
Locationabbr                  9295 non-null object
Locationdesc                  9295 non-null object
Class                         9295 non-null object
Topic                         9295 non-null object
Question                      9295 non-null object
Response                      9295 non-null object
Break_Out                     9295 non-null object
Break_Out_Category            9295 non-null object
Sample_Size                   9295 non-null int64
Data_value                    9278 non-null float64
Confidence_limit_Low          8940 non-null float64
Confidence_limit_High         8940 non-null float64
Display_order                 9295 non-null int64
Data_value_unit               9295 non-null object
Data_value_type               9295 non-null object
Data_Value_Footnote_Symbol    17 non-null object
Data_Value_Footnote         

# Merge risk factor data with census data

In [119]:
#merge the breweries count into a dataframe
census_and_risks = pd.merge(risk_df, all_census_data_df_with_states, how = 'left', left_on = ["Locationdesc"], right_on = ['State'])

In [120]:
class_cat = census_and_risks['Class'].nunique()
quest = census_and_risks['Question'].nunique()
topic = census_and_risks['Topic'].nunique()
responses = census_and_risks['Response'].nunique()
print("clases: ", class_cat, "topics: ", topic, "questions: ", quest, "responses: ", responses)

clases:  15 topics:  51 questions:  64 responses:  85


In [121]:
census_and_risks['combined'] = census_and_risks['Class'].astype(str) + "-" + census_and_risks['Topic'].astype(str) + "-" + census_and_risks['Question'].astype(str) + "-" + census_and_risks['Response'].astype(str) 

In [122]:
census_and_risks = census_and_risks[census_and_risks['Class'] != 'Demographics']

In [80]:
drop_cols = ['Class', 'Topic', 'Question', 'Response', 'Year', 'Sample_Size', 
            'Confidence_limit_Low', 'Confidence_limit_High',
            'Display_order', 'Data_Value_Footnote', 'Data_Value_Footnote_Symbol', 'Data_value_type',
            'DataSource', 'ClassId', 'TopicId', 'LocationID', 'BreakoutID', 
            'BreakOutCategoryID', 'QuestionID', 'ResponseID', 'FIPS', 'State',
            'Abbrev', 'GeoLocation', 'Break_Out', 'Break_Out_Category', 'Data_value_unit', 'Locationabbr']

In [81]:
census_and_risks.head()

,Year,Locationabbr,Locationdesc,Class,Topic,Question,Response,Break_Out,Break_Out_Category,Sample_Size,...,industry_professional_scientific_management,industry_education_health_care,industry_arts_entertatinment,industry_other_services,industry_public_admin,industry_military,FIPS,State,Abbrev,combined
5,2014,AL,Alabama,Alcohol Consumption,Alcohol Consumption,Adults who have had at least one drink of alco...,Yes,Overall,Overall,2887,...,186890.0,447613.0,166160.0,103169.0,112245.0,12132.0,01,Alabama,AL,Alcohol Consumption-Alcohol Consumption-Adults...
6,2014,AL,Alabama,Alcohol Consumption,Alcohol Consumption,Adults who have had at least one drink of alco...,No,Overall,Overall,5415,...,186890.0,447613.0,166160.0,103169.0,112245.0,12132.0,01,Alabama,AL,Alcohol Consumption-Alcohol Consumption-Adults...
7,2014,AL,Alabama,Alcohol Consumption,Binge Drinking,Binge drinkers (males having five or more drin...,Yes,Overall,Overall,677,...,186890.0,447613.0,166160.0,103169.0,112245.0,12132.0,01,Alabama,AL,Alcohol Consumption-Binge Drinking-Binge drink...
8,2014,AL,Alabama,Alcohol Consumption,Binge Drinking,Binge drinkers (males having five or more drin...,No,Overall,Overall,7553,...,186890.0,447613.0,166160.0,103169.0,112245.0,12132.0,01,Alabama,AL,Alcohol Consumption-Binge Drinking-Binge drink...
9,2014,AL,Alabama,Alcohol Consumption,Heavy Drinking,Heavy drinkers (adult men having more than two...,Yes,Overall,Overall,322,...,186890.0,447613.0,166160.0,103169.0,112245.0,12132.0,01,Alabama,AL,Alcohol Consumption-Heavy Drinking-Heavy drink...


In [82]:
census_and_risks.drop(drop_cols, axis = 1, inplace = True)
census_and_risks.reset_index(drop = True, inplace = True)

In [123]:
census_and_risks = census_and_risks[(census_and_risks['Locationdesc'] != 'Guam') 
                 & (census_and_risks['Locationdesc'] != 'All States and DC (median) **') 
                 & (census_and_risks['Locationdesc'] != 'All States, DC and Territories (median) **')]

In [124]:
census_and_risks.to_csv('census_and_risks.csv', index = False)

## Find a correlation in the data

In [99]:
combined_qs = census_and_risks['combined'].unique()

In [100]:
i = 0
for q in combined_qs:
    if i == 0:
        result = census_and_risks[census_and_risks['combined'] == q].corr()
        result = pd.DataFrame(result.iloc[0])
        result.rename(columns = {'Data_value': q}, inplace = True)
    else:
        result2 = census_and_risks[census_and_risks['combined'] == q].corr()
        result2 = pd.DataFrame(result2.iloc[0])
        result2.rename(columns = {'Data_value': q}, inplace = True)
        result = result.merge(result2, right_index = True, left_index = True)
    i += 1

In [101]:
result = result.transpose()

In [103]:
result.to_csv('result.csv')

## Create CSV with only the highly correlated data

* Tobacco Use-Smoker Status-Four Level Smoking Status (variable calculated from one or more BRFSS questions)-Former smoker
* median_age_male

In [125]:
census_and_risks2 = pd.read_csv('census_and_risks.csv')

In [126]:
target = census_and_risks2[census_and_risks2['Class'] == "Tobacco Use"].copy()
target = target[(target['Topic'] == "Smoker Status") & (target['Response'] == "Former smoker")]

In [129]:
target.columns

Index(['Year', 'Locationabbr', 'Locationdesc', 'Class', 'Topic', 'Question',
       'Response', 'Break_Out', 'Break_Out_Category', 'Sample_Size',
       'Data_value', 'Confidence_limit_Low', 'Confidence_limit_High',
       'Display_order', 'Data_value_unit', 'Data_value_type',
       'Data_Value_Footnote_Symbol', 'Data_Value_Footnote', 'DataSource',
       'ClassId', 'TopicId', 'LocationID', 'BreakoutID', 'BreakOutCategoryID',
       'QuestionID', 'ResponseID', 'GeoLocation', 'tot_pop', 'male_pop',
       'female_pop', 'median_age', 'median_age_male', 'median_age_female',
       'employ_status_tot', 'employ_status_working',
       'employ_status_working_civ', 'employ_status_working_civ_employed',
       'employ_status_working_civ_unemployed',
       'employ_status_working_military', 'employ_status_not_working',
       'birth_ed_tot', 'birth_ed_less_than_high', 'birth_ed_high',
       'birth_ed_some_coll', 'birth_ed_batchelors', 'birth_ed_graduate',
       'birth_income_tot', 'birth_inc

In [130]:
columns = ['Locationabbr', 'median_age_male', 'Data_value']

In [132]:
target = target[columns].reset_index(drop = True)

In [133]:
target.head()

,Locationabbr,median_age_male,Data_value
0,AL,37.1,24.6
1,AK,33.3,25.7
2,AZ,35.8,26.5
3,AR,36.4,25.2
4,CA,34.9,22.8


In [134]:
target.to_csv('target.csv', index = False)